In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
from openai import OpenAI
import json

def create_client():
    return OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

def setup_content(location: str, preferences: str):
    return f"I am traveling to {location}, and I want to visit attractions around this area that fit these criteria: {preferences}. Generate a list of 10 attraction objects and return each attraction name and entire address" 

def get_response(client, content: str):
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant designed to output JSON containing a list of objects that have atrribute name and address."},
            {"role": "user", "content": content,}
        ],
    )

def get_locations(location: str, preferences: str = None):

    client = create_client()
    chat_completion = get_response(client,  setup_content(location, preferences))

    return json.loads(chat_completion.choices[0].message.content)["attractions"]

<h2>Melissa Functions</h2>

In [29]:
import xmltodict
import urllib.request, urllib.parse

def make_url(address: str):
    melissa_url = "https://address.melissadata.net/v3/WEB/GlobalAddress/doGlobalAddress?" + urllib.parse.urlencode([('id', os.environ.get("MELISSA_API_KEY")), ('opt', "OutputGeo:ON"), ('a1', address)])
    print(melissa_url)
    return melissa_url

def make_request(url: str):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    if response.getcode() == 200:
        return response.read().decode(encoding = 'utf-8')

def get_coords(location: str) -> tuple:
    req = make_request(make_url(location))
    response = xmltodict.parse(req)
    try:
        latitude = response['Response']['Records']['ResponseRecord']['Latitude']
        longitude = response['Response']['Records']['ResponseRecord']['Longitude']
    except KeyError:
        print("Invalid address")
        latitude = None
        longitude = None
    return (latitude, longitude)
    
def manhattan_distance(lat1, lon1, lat2, lon2):
    lat_diff = abs(lat1 - lat2)
    lon_diff = abs(lon1 - lon2)
    return lat_diff + lon_diff


In [30]:
# locations = get_locations("UC Irvine", "nature")
for location in locations:
    print(location['address'])
    coords = get_coords(location['address'])
    location["lat"], location["lon"] = coords[0], coords[1]

print(locations)


5 Riparian View, Irvine, CA 92612, United States
https://address.melissadata.net/v3/WEB/GlobalAddress/doGlobalAddress?id=h7JJYwkNrxXkA7DLt0ebaU%2A%2AnSAcwXpxhQ0PC2lXxuDAZ-%2A%2A&opt=OutputGeo%3AON&a1=5+Riparian+View%2C+Irvine%2C+CA+92612%2C+United+States
Invalid address
18712 University Dr, Irvine, CA 92612, United States
https://address.melissadata.net/v3/WEB/GlobalAddress/doGlobalAddress?id=h7JJYwkNrxXkA7DLt0ebaU%2A%2AnSAcwXpxhQ0PC2lXxuDAZ-%2A%2A&opt=OutputGeo%3AON&a1=18712+University+Dr%2C+Irvine%2C+CA+92612%2C+United+States
Invalid address
Shady Canyon, Irvine, CA, United States
https://address.melissadata.net/v3/WEB/GlobalAddress/doGlobalAddress?id=h7JJYwkNrxXkA7DLt0ebaU%2A%2AnSAcwXpxhQ0PC2lXxuDAZ-%2A%2A&opt=OutputGeo%3AON&a1=Shady+Canyon%2C+Irvine%2C+CA%2C+United+States
Invalid address
34 Shady Canyon Dr, Irvine, CA 92603, United States
https://address.melissadata.net/v3/WEB/GlobalAddress/doGlobalAddress?id=h7JJYwkNrxXkA7DLt0ebaU%2A%2AnSAcwXpxhQ0PC2lXxuDAZ-%2A%2A&opt=OutputGeo%3A